In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_5_only_3.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_5_only_3.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,CH,CH,16.0
1,GB,GB,832.0
2,US,US,13503.0
3,CA,CA,409.0
4,AU,AU,214.0
...,...,...,...
577,PT,NG,1.0
578,UG,US,1.0
579,GB,TR,1.0
580,AG,US,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

15674.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,16.0,,,,
1,GB,GB,832.0,,,,
2,US,US,13503.0,,,,
3,CA,CA,409.0,,,,
4,AU,AU,214.0,,,,
...,...,...,...,...,...,...,...
577,PT,NG,1.0,,,,
578,UG,US,1.0,,,,
579,GB,TR,1.0,,,,
580,AG,US,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,16.0,99.0,99.0,69.0,30.0
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0
4,AU,AU,214.0,807.0,807.0,382.0,425.0
...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0
578,UG,US,1.0,6.0,30928.0,4.0,15254.0
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0
580,AG,US,1.0,8.0,30928.0,3.0,15254.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,16.0,99.0,99.0,69.0,30.0,,
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0,,
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0,,
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0,,
4,AU,AU,214.0,807.0,807.0,382.0,425.0,,
...,...,...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0,,
578,UG,US,1.0,6.0,30928.0,4.0,15254.0,,
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0,,
580,AG,US,1.0,8.0,30928.0,3.0,15254.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,16.0,99.0,99.0,69.0,30.0,53.333333,23.188406
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0,63.125948,58.881812
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0,88.521044,86.149037
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0,55.345061,59.534207
4,AU,AU,214.0,807.0,807.0,382.0,425.0,50.352941,56.020942
...,...,...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0,0.5,16.666667
578,UG,US,1.0,6.0,30928.0,4.0,15254.0,0.006556,25.0
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0,7.142857,0.070771
580,AG,US,1.0,8.0,30928.0,3.0,15254.0,0.006556,33.333333


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,16.0,99.0,99.0,69.0,30.0,53.333333,23.188406,,,
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0,63.125948,58.881812,,,
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0,88.521044,86.149037,,,
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0,55.345061,59.534207,,,
4,AU,AU,214.0,807.0,807.0,382.0,425.0,50.352941,56.020942,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0,0.5,16.666667,,,
578,UG,US,1.0,6.0,30928.0,4.0,15254.0,0.006556,25.0,,,
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0,7.142857,0.070771,,,
580,AG,US,1.0,8.0,30928.0,3.0,15254.0,0.006556,33.333333,,,


In [19]:
import pypopulation

In [20]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [21]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,16.0,99.0,99.0,69.0,30.0,53.333333,23.188406,0.000001,,
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0,63.125948,58.881812,0.000006,,
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0,88.521044,86.149037,0.000021,,
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0,55.345061,59.534207,0.000005,,
4,AU,AU,214.0,807.0,807.0,382.0,425.0,50.352941,56.020942,0.000004,,
...,...,...,...,...,...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0,0.5,16.666667,0.0,,
578,UG,US,1.0,6.0,30928.0,4.0,15254.0,0.006556,25.0,0.0,,
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0,7.142857,0.070771,0.0,,
580,AG,US,1.0,8.0,30928.0,3.0,15254.0,0.006556,33.333333,0.0,,


In [22]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [23]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,16.0,99.0,99.0,69.0,30.0,53.333333,23.188406,0.000001,0.080808,0.161616
1,GB,GB,832.0,2731.0,2731.0,1413.0,1318.0,63.125948,58.881812,0.000006,0.152325,0.30465
2,US,US,13503.0,30928.0,30928.0,15674.0,15254.0,88.521044,86.149037,0.000021,0.218297,0.436595
3,CA,CA,409.0,1426.0,1426.0,687.0,739.0,55.345061,59.534207,0.000005,0.143408,0.286816
4,AU,AU,214.0,807.0,807.0,382.0,425.0,50.352941,56.020942,0.000004,0.13259,0.26518
...,...,...,...,...,...,...,...,...,...,...,...,...
577,PT,NG,1.0,18.0,334.0,6.0,200.0,0.5,16.666667,0.0,0.002841,0.004854
578,UG,US,1.0,6.0,30928.0,4.0,15254.0,0.006556,25.0,0.0,0.000032,0.000066
579,GB,TR,1.0,2731.0,25.0,1413.0,14.0,7.142857,0.070771,0.0,0.000363,0.000701
580,AG,US,1.0,8.0,30928.0,3.0,15254.0,0.006556,33.333333,0.0,0.000032,0.000066


In [24]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_5_only_3.csv', index=False)